### SVM Gaussian kernel visualization (hard margins)

Author: Teun Mathijssen

You can choose either of two datasets: XOR or circular.

In [ ]:
%pylab inline

import cvxopt
from mpl_toolkits.mplot3d import Axes3D
cvxopt.solvers.options['show_progress'] = False

In [ ]:
def create_X_and_t(X1, X2):
    """Concatenate points and construct a list of labels for both clusters."""
    X = np.vstack([X1, X2])
        
    t_1 = np.ones((X1.shape[0])) * -1
    t_2 = np.ones((X2.shape[0]))
    
    t = np.concatenate([t_1, t_2])
    
    return X, t

In [ ]:
def construct_K(X, t, s):
    "Construct the kernel matrix K using Gaussian kernel functions."
    N_test = len(X)
    
    K = np.zeros((N_test, N_test))
    for n in range(N_test):
        for m in range(N_test):
            K[n][m] = t[n]*t[m]*k(X[n], X[m], s)
    
    return K

def k(x, x_p, s=4):
    """Gaussian kernel: Bishop Eq. 6.23"""
    return np.exp(-(x - x_p)@(x - x_p) / (2*s**2))

def kernel_trick(X, X_sup):
    "Perform the kernel trick for a dataset X."
    N_sup = X_sup.shape[0]
    sum_n = np.zeros(X.shape[0])
    
    # Evaluate the kernel k(X_sup, X) for all support vectors X_sup_n against all points in X.
    for n in range(N_sup):
        for i in range(X.shape[0]):
                sum_n[i] += a_sup[n]*t_sup[n]*k(X_sup[n], X[i], s)
    
    return sum_n + b

In [ ]:
def compute_multipliers(X, t, s):
    """From the lab exercise."""
    N_test = X.shape[0]
    
    P = cvxopt.matrix(construct_K(X, t, s))
    q = cvxopt.matrix(-np.ones((N_test, 1)))
    
    G = cvxopt.matrix(-1*np.eye(N_test))
    h = cvxopt.matrix(np.zeros(N_test))
    
    A = cvxopt.matrix(t[np.newaxis])
    b = cvxopt.matrix(np.zeros(1))

    sol = cvxopt.solvers.qp(P, q, G, h, A, b)
    a = np.array(sol['x'])
    
    return np.squeeze(a)

In [ ]:
### Generate XOR dataset using 4 Gaussian clusters.
mu_1 = np.array([-1, -1])
mu_2 = np.array([-1, 1])

Sigma_1 = np.eye(2) * 0.09
Sigma_2 = np.eye(2) * 0.09

# Number of points per cluster location.
N_1 = 20
N_2 = 20

X_1 = np.vstack([np.random.multivariate_normal(mu_1 * -1, Sigma_1, N_1), np.random.multivariate_normal(mu_1, Sigma_1, N_1)])
X_2 = np.vstack([np.random.multivariate_normal(mu_2 * -1, Sigma_2, N_2), np.random.multivariate_normal(mu_2, Sigma_2, N_2)])

### Also try the circular dataset! Just uncomment this.
# X_1 = np.random.multivariate_normal(np.zeros(2), Sigma_1, N_1*2)
# angles = np.random.uniform(0, 2*np.pi, N_2*2)
# radiuses = 1.8 + np.random.normal(0, 0.15, N_2*2)
# X_2 = np.array([radiuses*np.cos(angles), radiuses*np.sin(angles)]).T

X, t = create_X_and_t(X_1, X_2)

fig = plt.figure(figsize=(12, 6))

plt.scatter(X_1[:, 0], X_1[:, 1], color="red", s=25, marker="x", )
plt.scatter(X_2[:, 0], X_2[:, 1], color="blue", s=25, marker="x", )
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.title("Plot of XOR dataset")
plt.show()

In [ ]:
# Use sigma = 1 because it works well :)
s = 1
a = compute_multipliers(X, t, s)

# Epsilon to extract support vectors (SVs).
eps = 1e-5
sup_idx = a > eps
N_sup = np.sum(sup_idx)

# "weight" of the SVs.
a_sup = a[sup_idx]
# Class labels of the SVs.
t_sup = t[sup_idx]
# SVs.
X_sup = X[sup_idx]

# Calculate b using the set of SVs.
sum_n = 0
for n in range(N_sup):
    sum_m = 0
    for m in range(N_sup):
        sum_m += a_sup[m]*t_sup[m]*k(X_sup[n], X_sup[m], s)
        
    sum_n += t_sup[n] - sum_m
    
b = sum_n / N_sup

# Construct a coordinate grid (for plotting) and obtain the kernelized values of each grid point.
pad = 1
N_points = 100
x1 = np.linspace(np.min(X[:, 0]) - pad, np.max(X[:, 0]) + pad, N_points)
x2 = np.linspace(np.min(X[:, 1]) - pad, np.max(X[:, 1]) + pad, N_points)
X1, X2 = np.meshgrid(x1, x2)
# Reshape to convert meshgrid to list for kernel trick function.
gridspace = np.dstack([X1, X2]).reshape((-1, 2))
kernelspace = kernel_trick(gridspace, X_sup)
# Reshape the resulting list back to grid shape for proper plotting.
Y = kernelspace.reshape(X1.shape)

In [ ]:
plt.figure(figsize=(12, 6))

# Show zero level set.
plt.contour(X1, X2, Y, [0], colors="black")
# Show -1 and 1 level set.
plt.contour(X1, X2, Y, [-1, 1], colors="black", linestyles="dotted")
# Show the general contour of y(x).
plt.contour(X1, X2, Y, 9, colors="black", alpha=0.1, linestyles="-", linewidths=1)

# Color the decision regions.
cs = plt.contourf(X1, X2, Y, [-1, 1], extend="both", alpha=0.1, cmap="RdBu", vmin=-1, vmax=1)
cs.cmap.set_over('blue')
cs.cmap.set_under('red')

plt.scatter(X_1[:, 0], X_1[:, 1], color="red", s=25, marker="x", label="$X_1$")
plt.scatter(X_2[:, 0], X_2[:, 1], color="blue", s=25, marker="x", label="$X_2$")
plt.scatter(X_sup[:, 0], X_sup[:, 1], color="lime", s=150, facecolors="none", linewidths=2, label="support vector")

plt.xlabel("x1")
plt.ylabel("x2")
plt.title("Showing the decision boundary and support vectors in 2D (from directly above)")
plt.legend()

plt.show()

In [ ]:
Y_scale = np.min([np.max(Y), np.abs(np.min(Y))])
Y_X_1 = kernel_trick(X_1, X_sup)
Y_X_2 = kernel_trick(X_2, X_sup)
Y_X_sup = kernel_trick(X_sup, X_sup)

fig = plt.figure(figsize=(16, 11))

ax = fig.add_subplot(111, projection="3d")
ax.plot_surface(X1, X2, Y, alpha=0.10, cmap='RdBu', vmin=-Y_scale, vmax=Y_scale)
ax.contour(X1, X2, Y, [0], alpha=1, colors="black", linewidths=1)
ax.contour(X1, X2, Y, [-1, 1], alpha=1, colors="black", linestyles="dotted")

ax.scatter(X_1[:, 0], X_1[:, 1], Y_X_1, color="red", s=40, marker="x", label="$X_1$")
ax.scatter(X_2[:, 0], X_2[:, 1], Y_X_2, color="blue", s=40, marker="x", label="$X_2$")
ax.scatter(X_sup[:, 0], X_sup[:, 1], Y_X_sup, color="lime", s=150, facecolors="none", linewidths=2, label="support vector")

ax.view_init(elev=0, azim=-75)

ax.set_xlabel("x1")
ax.set_ylabel("x2")
ax.set_zlabel("y(x)")
plt.suptitle("Showing the decision landscape and support vectors from the side")

plt.show()

In [ ]:
fig = plt.figure(figsize=(16, 11))
ax = fig.add_subplot(111, projection="3d")

Y_above = np.where(Y > 0, Y, 0)

ax.contour(X1, X2, Y_above, 12, alpha=0.5, cmap='RdBu', vmin=-Y_scale, vmax=Y_scale)
ax.contour(X1, X2, Y, [0], alpha=1, colors="black", linewidths=1)
ax.contour(X1, X2, Y, [1], alpha=1, colors="black", linestyles="--")
ax.plot_wireframe(X1, X2, 0, alpha=0.15, color="black", linewidths=1)

ax.scatter(X_1[:, 0], X_1[:, 1], Y_X_1, color="red", s=40, marker="x", label="$X_1$")
ax.scatter(X_2[:, 0], X_2[:, 1], Y_X_2, color="blue", s=40, marker="x", label="$X_2$")
ax.scatter(X_sup[:, 0], X_sup[:, 1], Y_X_sup, color="lime", s=150, facecolors="none", linewidths=2, label="support vector")

ax.view_init(elev=20, azim=-90)

ax.set_xlabel("x1")
ax.set_ylabel("x2")
ax.set_zlabel("y(x)")
plt.suptitle("Showing the decision landscape and support vectors angled from above")
    
plt.show()

In [ ]:
fig = plt.figure(figsize=(16, 11))
ax = fig.add_subplot(111, projection="3d")

Y_below = np.where(Y < 0, Y, 0)

ax.contour(X1, X2, Y_below, 12, alpha=0.5, cmap='RdBu', vmin=-Y_scale, vmax=Y_scale)
ax.contour(X1, X2, Y, [0], alpha=1, colors="black", linewidths=1)
ax.contour(X1, X2, Y, [-1], alpha=1, colors="black", linestyles="--")
ax.plot_wireframe(X1, X2, 0, alpha=0.15, color="black", linewidths=1)

ax.scatter(X_1[:, 0], X_1[:, 1], Y_X_1, color="red", s=40, marker="x", label="$X_1$")
ax.scatter(X_2[:, 0], X_2[:, 1], Y_X_2, color="blue", s=40, marker="x", label="$X_2$")
ax.scatter(X_sup[:, 0], X_sup[:, 1], Y_X_sup, color="lime", s=150, facecolors="none", linewidths=2, label="support vector")

ax.view_init(elev=-20, azim=-90)

ax.set_xlabel("x1")
ax.set_ylabel("x2")
ax.set_zlabel("y(x)")
plt.suptitle("Showing the decision landscape and support vectors angled from below")
    
plt.show()